In [3]:
import re
import tweepy
import spacy
import pandas as pd
import plotly.express as px
import streamlit as st
from transformers import pipeline

C:\Users\tramy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
st.title("AI-Powered Social Media Sentiment Analysis for Brands")
st.write("Enter a keyword or brand name to analyze sentiment on Twitter.")

2025-03-06 14:18:07.260 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 14:18:07.771 
  command:

    streamlit run C:\Users\tramy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-06 14:18:07.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 14:18:07.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 14:18:07.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 14:18:07.771 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-06 14:18:07.771 Thread 'MainThread': missing ScriptRunContext! This w

In [5]:
# Twitter API credentials (replace with your own credentials)
api_key = "your_api_key"
api_secret_key = "your_api_secret_key"
access_token = "your_access_token"
access_token_secret = "your_access_token_secret"

# Authenticate with Twitter API
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [6]:
# Load spaCy model for text preprocessing
nlp = spacy.load("en_core_web_sm")

# Load sentiment analysis pipeline (RoBERTa model)
sentiment_pipeline = pipeline("sentiment-analysis", model="roberta-base")

# Function to fetch tweets
def fetch_tweets(query, count=100):
    try:
        tweets = api.search_tweets(q=query, count=count, tweet_mode="extended")
        return [tweet.full_text for tweet in tweets]
    except tweepy.TweepError as e:
        st.error(f"Error fetching tweets: {e}")
        return []

# Function to clean tweets
def clean_tweet(tweet):
    tweet = re.sub(r"http\S+|www\S+|https\S+", "", tweet)  # Remove URLs
    tweet = re.sub(r"\@\w+|\#", "", tweet)  # Remove mentions and hashtags
    tweet = re.sub(r"[^\w\s]", "", tweet)  # Remove special characters
    return tweet.strip()

# Function to preprocess tweets
def preprocess_tweet(tweet):
    doc = nlp(tweet)
    tokens = [token.lemma_ for token in doc if not token.is_stop]
    return " ".join(tokens)

# Function to analyze sentiment of a tweet
def analyze_sentiment(tweet):
    result = sentiment_pipeline(tweet)
    return result[0]["label"]

# Function to aggregate sentiment results
def aggregate_sentiments(tweets):
    sentiments = [analyze_sentiment(tweet) for tweet in tweets]
    sentiment_counts = pd.Series(sentiments).value_counts().reset_index()
    sentiment_counts.columns = ["Sentiment", "Count"]
    return sentiment_counts

# Function to visualize sentiment distribution
def plot_sentiment_distribution(sentiment_counts):
    fig = px.pie(sentiment_counts, values="Count", names="Sentiment", title="Sentiment Distribution")
    st.plotly_chart(fig)

# Function to display individual tweets and their sentiment
def display_tweets_with_sentiment(tweets):
    st.subheader("Individual Tweets and Sentiment")
    tweet_data = []
    for tweet in tweets:
        sentiment = analyze_sentiment(tweet)
        tweet_data.append({"Tweet": tweet, "Sentiment": sentiment})
    tweet_df = pd.DataFrame(tweet_data)
    st.dataframe(tweet_df)

# Main workflow
def main():
    # User input
    user_input = st.text_input("Enter a keyword or brand name:")
    if st.button("Analyze"):
        if user_input:
            st.write(f"Analyzing tweets for: {user_input}")

            # Fetch tweets
            tweets = fetch_tweets(user_input)
            if not tweets:
                st.warning("No tweets found. Please try a different keyword.")
                return

            # Preprocess tweets
            cleaned_tweets = [clean_tweet(tweet) for tweet in tweets]
            preprocessed_tweets = [preprocess_tweet(tweet) for tweet in cleaned_tweets]

            # Analyze sentiment
            sentiment_counts = aggregate_sentiments(preprocessed_tweets)

            # Display sentiment distribution
            st.subheader("Sentiment Analysis Results")
            plot_sentiment_distribution(sentiment_counts)

            # Display individual tweets and their sentiment
            display_tweets_with_sentiment(preprocessed_tweets)
        else:
            st.warning("Please enter a keyword or brand name.")

# Run the app
if __name__ == "__main__":
    main()

C:\Users\tramy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tramy\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


RuntimeError: Failed to import transformers.models.roberta.modeling_tf_roberta because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.